# CLEAN LOAN CERTIFICATES

In [1]:
import numpy as np
import pandas as pd
import math
import spacy
import re

In [2]:
delegates_og = pd.read_excel("Data/Delegates/constitutional_convention_1787.xlsx", header = 2)
delegates = delegates_og.copy()

In [3]:
#fix second last name in parentheses
#fitzsimons and fitzSimons are the same because we dont care abt caps

In [4]:
print("dimensions:", delegates.shape)
last_name_2 = delegates[delegates['last name'].apply(lambda x: "(" in x)].index
delegates[delegates['last name'].apply(lambda x: "(" in x)]

dimensions: (55, 4)


,first name,last name,state,sign?
2,Oliver,Ellsworth (Elsworth),Connecticut,no
24,David,Brearly (Brearley),New Jersey,yes
25,William,Paterson (Patterson),New Jersey,yes
40,Thomas,Fitzsimons (FitzSimons; Fitzsimmons),Pennsylvania,yes


In [5]:
#cleaning parentheses
delegates['last name 2'] = pd.Series()
delegates['last name'].loc[40] = "Fitzsimons (Fitzsimmons)"
delegates['last name 2'].loc[last_name_2] = delegates['last name'][last_name_2].apply(lambda x: x[x.find("(")+1:x.find(")")])
delegates['last name 2'].loc[last_name_2]

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


2        Elsworth
24       Brearley
25      Patterson
40    Fitzsimmons
Name: last name 2, dtype: object

In [6]:
delegates = delegates[['first name', 'last name', 'last name 2', 'state', 'sign?']]
#delegates

## Handle Loan Office Certificates from the 9 States

In [7]:
loans_og = pd.read_excel("Data/Pre1790/loan_office_certificates_9_states.xlsx")
loans = loans_og.copy()

In [8]:
loans['Title 3'] = pd.Series()
loans['First Name 3'] = pd.Series()
loans['Last Name 3'] = pd.Series()

In [9]:
loans['notes'] = pd.Series()
loans['original text'] = pd.Series()

In [10]:
loans.columns

Index(['State', 'Year', 'Month', 'Day', 'Title 1', 'First Name 1 ',
       'Last Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2', 'Face Value',
       'Specie Value ', 'Title 3', 'First Name 3', 'Last Name 3', 'notes',
       'original text'],
      dtype='object')

In [11]:
loans = loans[list(loans.columns[:10]) + list(loans.columns[12:15]) + list(loans.columns[10:12]) + ['notes','original text']]

In [12]:
def betterIsNan(x):
    try: 
        return math.isnan(x)
    except:
        return False

In [13]:
print("checking if there are any columns with valid last name 1 and first name 2")
any([betterIsNan(x) and betterIsNan(y) and not betterIsNan(w) and not betterIsNan(z) for w, x, y, z in zip(loans['First Name 1 '], loans['Last Name 1 '], loans['First Name 2'], loans['Last Name 2'])])

checking if there are any columns with valid last name 1 and first name 2


False

In [14]:
print("checking if there are any columns with valid last name 2 and first name 1")
any([betterIsNan(w) and betterIsNan(z) and not betterIsNan(x) and not betterIsNan(y) for w, x, y, z in zip(loans['First Name 1 '], loans['Last Name 1 '], loans['First Name 2'], loans['Last Name 2'])])

checking if there are any columns with valid last name 2 and first name 1


False

In [15]:
def replaceStrings(x):
    try:
        return x.replace("  ", " ")
    except:
        return x

In [16]:
loans['First Name 1 '] = loans['First Name 1 '].apply(lambda x: replaceStrings(x))

In [17]:
# get rid of entries where there is nothing for first name 1/2, last name 1/2 

In [18]:
no_first_name = loans[loans['First Name 1 '].apply(betterIsNan)]
#no_first_name

In [19]:
remove_indices = dict()
remove_indices['no_first_name'] = no_first_name[no_first_name['Last Name 1 '].apply(betterIsNan)].index.tolist()
#no_first_name.loc[remove_indices['no_first_name']]

In [20]:
#last name column is null
no_last_name = loans[loans['Last Name 1 '].apply(betterIsNan)]
#removing overlaps with the no first name column
no_last_name = no_last_name.loc[[x not in set(no_last_name.index).intersection(no_first_name.index) for x in no_last_name.index]]
#no_last_name

In [21]:
def lenGreaterOne(x):
    try: 
        return len(x.split(" ")) > 1 
    except: 
        return False

In [22]:
#clean entries where last name 1 is missing, first name 1 is only one word

In [23]:
one_word_first = no_last_name[no_last_name['First Name 1 '].apply(lambda x: not lenGreaterOne(x))]
remove_indices['one_word_first'] = one_word_first.index.tolist()
#one_word_first.loc[remove_indices['one_word_first']]

In [24]:
# clean entries where there is no last name and the first name is more than one character - follow steps below

In [25]:
weird_first_names = no_last_name[no_last_name['First Name 1 '].apply(lambda x: lenGreaterOne(x))]

<h2> Handle Corrections of Executors </h2>

How do we want to consider executors? Are they irrelevant (ie: do we only care about the person who they are executing for, or do we also care about them - and just add a note saying "executor")? 

Empirically, what interest do the executors have? They probably don't have any direct financial interest in the debt but there could definitely be incentives for them to be interested in the stability of the debt, such as shared business interests, desire to properly help execute a will etc?

What do we do when we cannot properly identify executors? Or trustees?

In [26]:
#I use the title executor to generaly indicate someone keeping money for another - could also be trustee etc

In [27]:
#trend below - if its Executors just get Atkinsons
#if its David Mackey and Debt Executor just do David Mackey
#for jacob lentz and phillip fisher lentz I kept jacob fisher and removed philip lentz
#there are records for jacob fisher being a soldier but no records for philip lentz
#otherwise find the word "to" or "of" and get the name right after that

In [28]:
executor = weird_first_names[weird_first_names['First Name 1 '].apply(lambda x: "exec" in x.lower() or "executor" in x.lower() or "ex." in x.lower())]
executor.shape

(40, 17)

In [29]:
#executor['First Name 1 '].unique().tolist()

In [30]:
for x in executor.loc[[39712, 39713, 40030]]['First Name 1 ']:
    print(x)

Robert Morris and John Simon executor to the estate of
Robert Morris and John Simon executor to the estate of
Robert Morris and John Simon executor to the estate of R Gerraty


In [31]:
executor_names = executor['First Name 1 ']
#handle special cases
executor_names = executor_names.replace("Atkinsons Executors", "Atkinsons")
executor_names = executor_names.replace("David Mackey and Debt Executor", "David Mackey")
executor_names = executor_names.replace("Jacob Fisher Philip Lentz Ex.", "Jacob Fisher")
executor_names = executor_names.replace("Robert Morris and John Simon executor to the estate of", "Robert Morris and John Simon executor to the estate of R Gerraty")
spec_status_names = ("David Mackey", "Atkinsons", "Jacob Fisher", "Robert Morris") 
#find string after the words to or of
post_of = executor_names.apply(lambda x: x[x.lower().find(' to '):] if x not in spec_status_names else x)
post_to = executor_names.apply(lambda x: x[x.lower().find(' of '):] if x not in spec_status_names else x)
#merge results
pre_processing_executor_names = pd.Series(x if len(x) >= len(y) else y for x, y in zip(post_of, post_to))
#remove stop words
pre_processing_executor_names = pre_processing_executor_names.apply(lambda x: x.replace("to", "").replace("To", "").replace("of", "").replace("Of"," ").strip())
pre_processing_executor_names = pre_processing_executor_names.apply(lambda x: x.replace("the", "").replace("  ", " ").strip())
pre_processing_executor_names.unique()

array(['Sam Hasting', 'B Winchester', 'Will Thomas Hubbard Esq deceased',
       'Benj Winchester', 'estate R Gerraty',
       'estate Jacob Carver Deceased', 'Stephen Boyd', 'David Mackey',
       'Leon Ampach', 'Conaard Teulps', 'Estale Ju Eighter',
       'Jacob Fisher', 'Mat Mease and Co.', 'Jacob Calvert',
       'Christ V. Cross', 'Atkinsons'], dtype=object)

In [32]:
nlp = spacy.load("en_core_web_sm")
executor_names = dict()
for entity in set(pre_processing_executor_names):
    doc = nlp(entity)
    if len(doc.ents) == 1:
        ent = doc.ents[0]
        if ent.label_ == "PERSON":
            executor_names[entity] = ent.text
        elif ent.label == "ORG":
            executor_names[entity] = ent.text
        else:
            executor_names[entity] = entity
    else:
        executor_names[entity] = entity

        executor_names['estate Jacob Carver Deceased'] = 'Jacob Carver'

In [33]:
#executor_names

In [34]:
executor_names['estate R Gerraty'] = 'R Gerraty'
executor_names['Will Thomas Hubbard Esq deceased'] = 'Will Thomas Hubbard'
executor_names['Mat Mease and Co.'] = 'Mat Mease'
post_processing_executor_names = [executor_names[x] for x in pre_processing_executor_names]
post_processing_executor_index = executor['First Name 1 '].index
#post_processing_executor_names

In [35]:
#changing recipient of all the money
executor['original text'] = executor['First Name 1 '].copy()
executor['First Name 1 '] = post_processing_executor_names
executor['Last Name 1 '] = [x.split(" ")[-1] for x in executor['First Name 1 ']]
executor['First Name 1 '] = [" ".join(x.split(" ")[:-1]) for x in executor['First Name 1 ']]
#executor

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [36]:
executor_names = executor['original text']
#handle special cases
executor_names = executor_names.replace("Atkinsons Executors", "")
executor_names = executor_names.replace("David Mackey and Debt Executor", "David Mackey")
executor_names = executor_names.replace("Jacob Fisher Philip Lentz Ex.", "Philip Lentz")
executor_names = executor_names.replace("Robert Morris and John Simon executor to the estate of", "Robert Morris and John Simon")
executor_names = executor_names.replace("William Allison Ex. Of Mat Mease and Co.", "William Allison")
spec_status_names = ("David Mackey", "", "Philip Lentz", "Robert Morris and John Simon", "William Allison") 
#find string after the words to or of
post_exec = executor_names.apply(lambda x: x[:x.lower().find(' exec')] if x not in spec_status_names else x)
post_exec_to = executor_names.apply(lambda x: x[:x.lower().find(' to')] if x not in spec_status_names else x)
#merge results
pre_processing_executors = pd.Series(x if len(x) < len(y) else y for x, y in zip(post_exec, post_exec_to)).apply(lambda x: x.replace("Ex.", "").strip())
#remove stop words
#pre_processing_executors

In [37]:
names1 = []
names2 = []
for name in pre_processing_executors:
    names = name.split("and")
    if len(names) > 1:
        names1.append(names[0].strip())
        names2.append(names[1].strip())
    else:
        names2.append(np.nan)
        names1.append(name.strip())
print(names1)
print(names2)

['Sam Austin', 'Benj Allen', 'Joseph Allen', 'William B', 'William B', 'William B', 'William B', 'William B', 'William B', 'William B', 'William B', 'William B', 'William B', 'To Allen', 'To Allen', 'Robert Morris', 'Robert Morris', 'Robert Morris', 'Isaac Roush', 'John Edwards', 'David Mackey', 'David Mackey', 'George Liller', 'George Kelcher', 'George Kelcher', 'John Penny Cacker', 'Philip Lentz', 'Philip Lentz', 'Philip Lentz', 'William Allison', 'William Allison', 'Isaac Rauch', 'Jacob Fisher', 'John Edwards', '', '', '', '', '', '']
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'John Simon', 'John Simon', 'John Simon', 'Mary Utrey', nan, nan, nan, nan, 'Peter Sherer', 'Peter Sherer', nan, nan, nan, nan, nan, nan, 'Mary Sallie', 'Chris Lentz', nan, nan, nan, nan, nan, nan, nan]


In [38]:
#changing recipient of all the money
executor['First Name 2'] = names1
executor['Last Name 2'] = [x.split(" ")[-1] if x != '' else x for x in executor['First Name 2'] ]
executor['First Name 2'] = [" ".join(x.split(" ")[:-1]) if x != '' else x for x in executor['First Name 2'] ]
executor['Title 2'] = ["executor" if x != '' else np.nan for x in executor['First Name 2']]

executor['First Name 3'] = names2
executor['Last Name 3'] = [x.split(" ")[-1]  if not pd.isnull(x) else x for x in executor['First Name 3']]
executor['First Name 3'] = [" ".join(x.split(" ")[:-1])  if not pd.isnull(x) else x for x in executor['First Name 3']]
executor['First Name 2'] = [" ".join(x.split(" ")[:-1]) if x != '' else x for x in executor['First Name 2'] ]
executor['Title 3'] = ["executor" if not pd.isnull(x) else np.nan for x in executor['First Name 3']]
#executor

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [39]:
executor['notes'][[x == "Jacob" and y == "Carver" for x, y in zip(executor['First Name 1 '], executor['Last Name 1 '])]] = "Deceased, Estate"
executor['notes'][[x == "R" and y == "Gerraty" for x, y in zip(executor['First Name 1 '], executor['Last Name 1 '])]] = "Estate"
executor['notes'][[x == "Will Thomas" and y == "Hubbard" for x, y in zip(executor['First Name 1 '], executor['Last Name 1 '])]] = "Deceased"

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:9114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exe

<h3> Modify weird_first_names to include corrections </h3>

In [40]:
#make adjustments here
weird_first_names.loc[executor.index] = executor

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [41]:
executor_cleaned = weird_first_names.loc[executor.index]
filter_out_names = executor_cleaned.index.tolist()
#executor_cleaned

In [42]:
#trend below - if its two and's deal with first case below
#if its two &'s deal with second case below
#otherwise just split the names

In [43]:
two_names = weird_first_names[weird_first_names['First Name 1 '].apply(lambda x: ("&" in x or " and " in x.lower()))]
new_index = [x not in filter_out_names for x in two_names.index]
two_names = two_names.loc[new_index]
two_names.shape

(2261, 17)

<h2> Handle corrections of names with weird combinators </h2> 

In [44]:
multiple_and = two_names[two_names['First Name 1 '].apply(lambda x: x.count('and') > 1)]
multiple_and['original text'] = multiple_and['First Name 1 ']
#multiple_and

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [45]:
#add sarah charman to name 3, executor
#add john cavott and jacob aivl as executors?? in name 3
#add oiriginal to the present string

In [46]:
multiple_and['First Name 1 '] = multiple_and['First Name 1 '].replace('Sarah Charman for the use of Gilbert Hammond and Cornelius Tommand', 
                                                                      'Gilbert Hammond and Cornelius Tommand')
multiple_and['First Name 1 '] = multiple_and['First Name 1 '].replace('John Cavott and Jacob Aivl for Moses Dichey and George Dukey', 
                                                                      'Moses Dichey and George Dukey')
multiple_and['First Name 1 '] = multiple_and['First Name 1 '].replace('Jn Nixon I M Nerbitt and Alexander Forster', 
                                                                      'Jn Nixon and I M Nerbitt and Alexander Forster')
multiple_and['First Name 1 '] = multiple_and['First Name 1 '].replace('Samuel Ely and Michael Gellington Esq and Co',
                                                                     'Samuel Ely and Michael Gellington Esq')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [47]:
multiple_and[multiple_and['First Name 1 '].apply(lambda x: x == 'Gilbert Hammond and Cornelius Tommand')]

,State,Year,Month,Day,Title 1,First Name 1,Last Name 1,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text
40857,6,1777,6,23.0,NaN,Gilbert Hammond and Cornelius Tommand,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600,600.0,NaN,Sarah Charman for the use of Gilbert Hammond a...


In [48]:
tempindex = multiple_and[multiple_and['First Name 1 '].apply(lambda x: x == 'Gilbert Hammond and Cornelius Tommand')].index
multiple_and.loc[tempindex, ['First Name 3', 'Last Name 3', 'Title 3']] = ['Sarah', 'Charman', 'executor']
tempindex = multiple_and[multiple_and['First Name 1 '].apply(lambda x: x == 'John Cavott and Jacob Aivl')].index
multiple_and.loc[tempindex, ['First Name 3', 'Last Name 3', 'Title 3']] = ['John', 'Cavott', 'executor']
multiple_and.loc[tempindex, 'notes'] = ['Jacob Aivl executor']

<h3> Fix by replacing in two_names, not superior weird_first_names </h3> 

In [49]:
two_names.loc[multiple_and.index] = multiple_and
multiple_and_cleaned = two_names.loc[multiple_and.index]
multiple_and_cleaned['original text'] = multiple_and_cleaned['First Name 1 ']
#multiple_and_cleaned

In [50]:
multiple_and_symbol = two_names[two_names['First Name 1 '].apply(lambda x: x.count('&') > 1)]
#executive decision made - nic jacobs is executor of peter brown
#add that nicholas jacobs is executor
multiple_and_symbol['original text'] = multiple_and_symbol['First Name 1 ']
multiple_and_symbol['First Name 1 '] = multiple_and_symbol['First Name 1 '].replace('Peter Brown & Nic Jacobs Ex & …….', 
                                                                                    'Peter Brown')
tempindex = multiple_and_symbol[multiple_and_symbol['First Name 1 '].apply(lambda x: x == 'Peter Brown')].index
multiple_and_symbol.loc[tempindex, ['First Name 2', 'Last Name 2', 'Title 2']] = ['Nic', 'Jacobs', 'executor']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [51]:
two_names.loc[multiple_and_symbol.index] = multiple_and_symbol
two_names.loc[multiple_and_symbol.index]

,State,Year,Month,Day,Title 1,First Name 1,Last Name 1,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text
57469,6,1779,8,14.0,NaN,Hre Rest & Ch & St Pr Chumess,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500,29.41667,NaN,Hre Rest & Ch & St Pr Chumess
57680,6,1779,9,2.0,NaN,Peter Brown,NaN,executor,Nic,Jacobs,NaN,NaN,NaN,400,22.13333,NaN,Peter Brown & Nic Jacobs Ex & …….


In [52]:
#checking to make sure no string has both "and" and "&" as split symbol
two_names[(two_names['First Name 1 '].apply(lambda x: "&" in x and " and " in x))]

,State,Year,Month,Day,Title 1,First Name 1,Last Name 1,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text


In [53]:
more_complicated = two_names[two_names['First Name 1 '].apply(lambda x: " in " in x or " of " in x or "to " in x or "for" in x)]
more_complicated_names = more_complicated['First Name 1 '].unique().tolist()

In [54]:
more_complicated['original text'] = more_complicated['First Name 1 ']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [55]:
print(len(more_complicated_names))
print(more_complicated_names)

30
['John Gray &Tho Dawes Fws.to Sarah Green', 'Jn Gray & Thomas Dawes Trustees to Sach Green', 'Nathaniel Appleton and other trustees of Judah Monis Legasy', 'John Barrett & Sons Trustees to Creditors of John Elsworth', 'Society for Relief of Poor Masters of ships widows and children', 'Corporation for the Relief of Poor and Distressed Presbyterian Ministers', 'Jeremiah Halsey and Sarah Gaston for the Estate of Gaston Dec', 'Wm Allison & Sam Caldwell Ex to the est of David Caldwell for children', 'Jacob Brush & Mary Wroop to the est of Jacob Carver Ded', 'Mathew Greer & Mathew Greer for the heirs of Thomas Jones Deceased', 'Joseph Jacket and Anthany Jacket Trustees of Presbyterian Congregation N town', 'Isaac Roush & Mary Bldney of Jacob Carver', 'Samuel Johnston Inna Hanna and I Johnston in trust for the Hierrs of Phil Johnston', 'Michele Shubart For Corporation of Michelle & Zion Churches', 'Corporation for relief of poor and distressed presbyterian ministers', 'Ed Keasby & J Bilber

In [56]:
#add notes saying that its an organization
#add trustees/people for - label as executor
#create dictionary corresponding to object that dictates how cases are tobe handled

In [57]:
#this code will be very long because its manual

In [58]:
tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x == 'John Gray &Tho Dawes Fws.to Sarah Green')].index
results = ['Sarah', 'Green', 'executor', 'John', 'Gray', 'executor', 'Tho', 'Dawes']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2', 'Title 3', 'First Name 3', 'Last Name 3']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x == 'Jn Gray & Thomas Dawes Trustees to Sach Green')].index
results = ['Sarah', 'Green', 'executor', 'Jn', 'Gray', 'executor', 'Thomas', 'Dawes']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2', 'Title 3', 'First Name 3', 'Last Name 3']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'Nathaniel Appleton and other trustees of Judah Monis Legasy')].index
results = ['Judah', 'Monis', 'legacy', 'executor', 'Nathaniel', 'Appleton']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'notes', 'Title 2', 'First Name 2', 'Last Name 2']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'John Barrett & Sons Trustees to Creditors of John Elsworth')].index
results = ['John', 'Elsworth', 'creditors', 'executor', 'John', 'Barrett']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'notes', 'Title 2', 'First Name 2', 'Last Name 2']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'Society for Relief of Poor Masters of ships widows and children')].index
results = ['organization', 'Society for Relief of Poor Masters of ships widows and children']
more_complicated.loc[tempindex, ['Title 1', 'First Name 1 ']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'Corporation for the Relief of Poor and Distressed Presbyterian Ministers')].index
results = ['organization', 'Corporation for the Relief of Poor and Distressed Presbyterian Ministers']
more_complicated.loc[tempindex, ['Title 1', 'First Name 1 ']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x == 'Jeremiah Halsey and Sarah Gaston for the Estate of Gaston Dec')].index
results = ['Gaston', 'Dec', 'estate', 'executor', 'Jeremiah', 'Halsey', 'executor', 'Sarah', ' ']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'notes', 'Title 2', 'First Name 2', 'Last Name 2', 'Title 3', 'First Name 3', 'Last Name 3']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'Wm Allison & Sam Caldwell Ex to the est of David Caldwell for children')].index
results = ['David', 'Caldwell', 'estate for children', 'executor', 'Sam', 'Caldwell', 'executor', 'Wm', 'Allison']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'notes', 'Title 2', 'First Name 2', 'Last Name 2', 'Title 3', 'First Name 3', 'Last Name 3']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'Jacob Brush & Mary Wroop to the est of Jacob Carver Ded')].index
results = ['Jacob', 'Carver', 'estate, deceased', 'executor', 'Jacob', 'Brush', 'executor', 'Mary', 'Wroop']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'notes', 'Title 2', 'First Name 2', 'Last Name 2', 'Title 3', 'First Name 3', 'Last Name 3']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==   'Mathew Greer & Mathew Greer for the heirs of Thomas Jones Deceased')].index
results = ['Thomas', 'Jones', 'heirs, deceased', 'executor', 'Mathew', 'Greer']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'notes', 'Title 2', 'First Name 2', 'Last Name 2']] = results

In [59]:
tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'Joseph Jacket and Anthany Jacket Trustees of Presbyterian Congregation N town')].index
results = ['organization', 'Presbyterian Congregation N town', 'executor', 'Joseph', 'Jacket', 'executor', 'Anthany', 'Jacket']
more_complicated.loc[tempindex, ['Title 1', 'First Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2', 'Title 3', 'First Name 3', 'Last Name 3']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'Isaac Roush & Mary Bldney of Jacob Carver')].index
results = ['Jacob', 'Carver', 'executor', 'Mary', 'Bldney', 'executor', 'Isaac', 'Roush']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2', 'Title 3', 'First Name 3', 'Last Name 3']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'Samuel Johnston Inna Hanna and I Johnston in trust for the Hierrs of Phil Johnston')].index
results = ['Phil', 'Johnston', 'heirs', 'executor', 'Samuel', 'Johnston', 'executor', 'Inna Hanna', 'Johnston']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'notes', 'Title 2', 'First Name 2', 'Last Name 2', 'Title 3', 'First Name 3', 'Last Name 3']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'Michele Shubart For Corporation of Michelle & Zion Churches')].index
results = ['organization', 'Corporation of Michelle & Zion Churches', 'executor', 'Michele', 'Shubart']
more_complicated.loc[tempindex, ['Title 1', 'First Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'Corporation for relief of poor and distressed presbyterian ministers')].index
results = ['organization', 'Corporation for relief of poor and distressed presbyterian ministers']
more_complicated.loc[tempindex, ['Title 1', 'First Name 1 ']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'Ed Keasby & J Bilber by the Ers of J Dickinson')].index
results = ['J', 'Dickinson', 'executor', 'Ed', 'Keasby', 'executor', 'J', 'Bilber']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2', 'Title 3', 'First Name 3', 'Last Name 3']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'Gen. Kelehner and Pr. Sheser Esq. to State of Conrad Scheele')].index
results = ['Conrad', 'Scheele', 'estate', 'executor', 'Gen.', 'Kelehner', 'executor', 'Pr. Sheser', 'Esq']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'notes', 'Title 2', 'First Name 2', 'Last Name 2', 'Title 3', 'First Name 3', 'Last Name 3']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'Andrew Hagenbach and Catherine Brobst Guardians of John Brobst')].index
results = ['John', 'Brobst', 'executor', 'Andrew', 'Hagenbach', 'executor', 'Catherine', 'Brobst']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2', 'Title 3', 'First Name 3', 'Last Name 3']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'Robert Patterson Guardin for Sarah and Mary Stewart')].index
results = ['Sarah', 'Stewart', 'Mary', 'Stewart', 'executor', 'Robert', 'Patterson']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'First Name 2', 'Last Name 2', 'Title 3', 'First Name 3', 'Last Name 3']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'Peter Knight & Sus Woodrow Ex to the Estate of H Woodrow')].index
results = ['H', 'Woodrow', 'estate', 'executor', 'Peter', 'Knight', 'executor', 'Sus', 'Woodrow']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'notes', 'Title 2', 'First Name 2', 'Last Name 2', 'Title 3', 'First Name 3', 'Last Name 3']] = results

In [60]:
tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'James Camble NO 7015 is not on file & in supposed to the number not delivered')].index
results = ['problem', 'James Camble NO 7015 is not on file & in supposed to the number not delivered']
more_complicated.loc[tempindex, ['Title 1', 'First Name 1 ']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'The rector of christ and st. peters churches')].index
results = ['organization', 'The rector of christ and st. peters churches']
more_complicated.loc[tempindex, ['Title 1', 'First Name 1 ']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'Jere Kalbey & S Gaston to the Estate John Gaston de')].index
results = ['John', 'Gaston de', 'estate', 'executor', 'Jere', 'Kalbey', 'executor', 'S', 'Gaston']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'notes', 'Title 2', 'First Name 2', 'Last Name 2', 'Title 3', 'First Name 3', 'Last Name 3']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==   'Jer Hals by & I Gaston Ex for the Estate of Jn Gaster')].index
results = ['Jn', 'Gaster', 'estate', 'executor', 'Jer', 'Hals', 'executor', 'I', 'Gaston']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'notes', 'Title 2', 'First Name 2', 'Last Name 2', 'Title 3', 'First Name 3', 'Last Name 3']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'John Smith and James McDonald Guardian of the Heirs of John Gibson')].index
results = ['John', 'Gibson', 'heirs', 'executor', 'John', 'Smith', 'executor', 'James', 'McDonald']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'notes', 'Title 2', 'First Name 2', 'Last Name 2', 'Title 3', 'First Name 3', 'Last Name 3']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'Ex of William Stadlerman and Pk Lickon Ex of William Stadlerman')].index
results = ['William', 'Stadlerman', 'executor', 'Pk', 'Lickon']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'Michael Schubert for St Michael and Zeus Corporation')].index
results = ['organization', 'St Michael and Zeus Corporation', 'executor', 'Michael', 'Schubert']
more_complicated.loc[tempindex, ['Title 1', 'First Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x == 'John Steinmetz for Corperetion of F Mich & Lions Cer')].index
results = ['organization', 'Corperetion of F Mich & Lions Cer', 'executor', 'John', 'Steinmetz']
more_complicated.loc[tempindex, ['Title 1', 'First Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x ==  'A and J J Caldwell for EvMitchell')].index
results = ['Ev', 'Mitchell', 'executor', 'A', 'Caldwell', 'executor', 'J J', 'Caldwell']
more_complicated.loc[tempindex, ['First Name 1 ', 'Last Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2', 'Title 3', 'First Name 3', 'Last Name 3']] = results

tempindex = more_complicated[more_complicated['First Name 1 '].apply(lambda x: x == 'Jon Steinmetz for St Michaels and Zions Cerpreatere')].index
results = ['organization', 'St Michaels and Zions Cerpreatere', 'executor', 'Jon', 'Steinmetz']
more_complicated.loc[tempindex, ['Title 1', 'First Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2']] = results

In [61]:
two_names.loc[more_complicated.index] = more_complicated
more_complicated_cleaned = two_names.loc[more_complicated.index]
#two_names.loc[more_complicated.index]

<h2> Handle corrections of and co. </h2>

In [62]:
has_co = two_names[two_names['First Name 1 '].apply(lambda x: x[-2:] == "Co" or "co." in x.lower() or " others " in x.lower() and not betterIsNan(x))]
has_co_names = pd.Series(has_co['First Name 1 '].unique())

In [63]:
has_co['original text'] = has_co['First Name 1 ']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [64]:
capt_sam_filter = [x == 'Capt Samuel Wilman and Co.' for x in has_co['First Name 1 ']]
has_co['First Name 1 '][capt_sam_filter] = "Samuel"
has_co['Last Name 1 '][capt_sam_filter] = "Wilman"
has_co['Title 1'][capt_sam_filter] = "Capt"

paris_bro_filter = [x == 'Paris Brothers and co.' for x in has_co['First Name 1 ']]
has_co['First Name 1 '][paris_bro_filter] = math.nan
has_co['Last Name 1 '][paris_bro_filter] = "Paris"
has_co['notes'][paris_bro_filter] = "brothers"

gs_dewint_filter = [x == 'GS.Dewint & Co' for x in has_co['First Name 1 ']]
has_co['Last Name 1 '][gs_dewint_filter] = "Dewint"
has_co['First Name 1 '][gs_dewint_filter] = "GS."

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the 

In [65]:
spec_status_names = ['Capt Samuel Wilman and Co.', 'Paris Brothers and co.', 'GS.Dewint & Co']
spec_status_first_names = ['Samuel', math.nan, 'GS.']
post_and_sym = has_co_names.apply(lambda x: x[:x.lower().find('&')] if x not in spec_status_names else x)
post_and = has_co_names.apply(lambda x: x[:x.lower().find('and')] if x not in spec_status_names else x)
#merge results
pre_processing_co_names = pd.Series(y if len(x) >= len(y) else y for x, y in zip(post_and_sym, post_and))
#remove stop words
pre_processing_co_names = pre_processing_co_names.apply(lambda x: x.replace("&", "").replace("Co.", "").replace("co."," ").strip())
pre_processing_co_names = pre_processing_co_names.apply(lambda x: x.replace(" C", "").replace(" other", "").strip())

In [66]:
replace_co = dict(pd.Series(pre_processing_co_names.tolist(), index = has_co_names))
del replace_co[spec_status_names[0]]
del replace_co[spec_status_names[1]]
del replace_co[spec_status_names[2]]

In [67]:
change_names = [x not in spec_status_first_names for x in has_co['First Name 1 ']]

In [68]:
has_co.loc[change_names, 'First Name 1 '] = has_co['First Name 1 '][change_names].apply(lambda x: replace_co[x])
has_co.loc[change_names, 'Last Name 1 '] = [x.split(" ")[-1] for x in has_co['First Name 1 '][change_names]]
has_co.loc[change_names, 'First Name 1 '] = [" ".join(x.split(" ")[:-1]) for x in has_co['First Name 1 '][change_names]]

In [69]:
two_names.loc[has_co.index] = has_co
two_names.loc[has_co.index]

,State,Year,Month,Day,Title 1,First Name 1,Last Name 1,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text
6559,2,1778,2,14.0,NaN,James,Swan,NaN,NaN,NaN,NaN,NaN,NaN,500,299.2986,NaN,James Swan & Co
7119,2,1778,2,25.0,NaN,James,Swan,NaN,NaN,NaN,NaN,NaN,NaN,600,347.1167,NaN,James Swan & Co
7120,2,1778,2,25.0,NaN,James,Swan,NaN,NaN,NaN,NaN,NaN,NaN,600,347.1167,NaN,James Swan & Co
7878,2,1778,4,27.0,NaN,James,Swan,NaN,NaN,NaN,NaN,NaN,NaN,600,265.3917,NaN,James Swan & Co
7879,2,1778,4,27.0,NaN,James,Swan,NaN,NaN,NaN,NaN,NaN,NaN,600,265.3917,NaN,James Swan & Co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80888,9,1779,8,1.0,NaN,Mark,Nephew,NaN,NaN,NaN,NaN,NaN,NaN,4000,245.3333,NaN,Mark Nephew & Co
80889,9,1779,8,1.0,NaN,Mark,Nephew,NaN,NaN,NaN,NaN,NaN,NaN,2400,147.2000,NaN,Mark Nephew & Co
80890,9,1779,8,1.0,NaN,Mark,Nephew,NaN,NaN,NaN,NaN,NaN,NaN,8500,521.3333,NaN,Mark Nephew & Co
80891,9,1780,7,18.0,NaN,Mark,Nephew,NaN,NaN,NaN,NaN,NaN,NaN,1600,40.0000,NaN,Mark Nephew & Co


<h2> Handling Heirs </h2>

In [70]:
def heirPresent(x):
    try:
        return "heir" in x.lower()
    except:
        return False

In [71]:
heirs = two_names[two_names['First Name 1 '].apply(lambda x: heirPresent(x))]
heirs_names = heirs['First Name 1 '].unique()

In [72]:
heirs

,State,Year,Month,Day,Title 1,First Name 1,Last Name 1,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text
40272,6,1777,5,8.0,NaN,Margarett Grant herself and heirs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500,500.0,NaN,NaN


In [73]:
heirs.loc[40272, 'First Name 1 '] = "Margaret"
heirs.loc[40272, 'Last Name 1 '] = "Grant"
heirs.loc[40272, 'notes'] = "heirs"

In [74]:
two_names.loc[heirs.index] = heirs
two_names.loc[heirs.index]

,State,Year,Month,Day,Title 1,First Name 1,Last Name 1,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text
40272,6,1777,5,8.0,NaN,Margaret,Grant,NaN,NaN,NaN,NaN,NaN,NaN,500,500.0,heirs,NaN


In [75]:
filter_out_names.extend(has_co.index.tolist())
filter_out_names.extend(heirs.index.tolist())
filter_out_names.extend(more_complicated.index.tolist())

In [76]:
filter_out_names = list(set(filter_out_names))

<h2> Handling names separated by & and "and" </h2>

In [77]:
and_sep_filter = [x not in filter_out_names for x in two_names.index]

In [78]:
def splitNames(x):
    x = x.replace(" And ", " and ")
    if "&" in x:
        return x.split("&")
    return x.split(" and ")

In [79]:
normal_join = two_names.loc[and_sep_filter]
normal_join['First Name 1 '].apply(lambda x: splitNames(x)).apply(lambda x: len(x)).value_counts()

2    1717
3       2
1       1
Name: First Name 1 , dtype: int64

In [80]:
normal_join['First Name 1 '].unique()

array(['Lee & Jones', 'Clark & Nightingale', 'Pitts & Call',
       'Bryant & Dennie', 'Hubbart & Greene', 'Hunt & Sherburne',
       'Jackson Tracy & Tracy', 'Otis & Andrews', 'Parker & Eustis',
       'Cushing & White', 'Riggs Dairs and Humphrey',
       'Henry McClellen and Henry', 'John Brinkerhoff & others',
       'Mercer and Skenk', 'John Stevens and William Burnett',
       'Forman & Hunt', 'Perrigrine & Van Ember', 'Ogden & Curti',
       'Stephen Ford and Canfield', 'Forman and Hunt',
       'Hendrick & Clears', 'Isaac & Peter Vandorn',
       'Hendrick & Jacob TenEyk', 'Prior & Joy', 'Benezet & Backman',
       'Gilbert Hammond and Cornelius Tommand', 'Simons & Henry',
       'Rotch and Jarvis', 'Moses Dichey and George Dukey',
       'Levy and Levy', 'Peter and Isaac Wikoff',
       'Peter And Isaac Wikoff', 'Milner and Haynes',
       'Michel and Zion Churches', 'Chalone and White', 'Charles & White',
       'Vance Caldwell and Vance', 'Jacob and Henry Geiger',
       'Let

In [81]:
normal_join['original text'] = normal_join['First Name 1 ']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [82]:
joinedNames = normal_join['First Name 1 '].apply(lambda x: splitNames(x))

In [83]:
joinedNames[joinedNames.apply(lambda x: len(x)>2)]

53730    [Jn Nixon, I M Nerbitt, Alexander Forster]
57469             [Hre Rest ,  Ch ,  St Pr Chumess]
Name: First Name 1 , dtype: object

In [84]:
print(len(normal_join['First Name 1 '].unique()))
normal_join['First Name 1 '].unique()

139


array(['Lee & Jones', 'Clark & Nightingale', 'Pitts & Call',
       'Bryant & Dennie', 'Hubbart & Greene', 'Hunt & Sherburne',
       'Jackson Tracy & Tracy', 'Otis & Andrews', 'Parker & Eustis',
       'Cushing & White', 'Riggs Dairs and Humphrey',
       'Henry McClellen and Henry', 'John Brinkerhoff & others',
       'Mercer and Skenk', 'John Stevens and William Burnett',
       'Forman & Hunt', 'Perrigrine & Van Ember', 'Ogden & Curti',
       'Stephen Ford and Canfield', 'Forman and Hunt',
       'Hendrick & Clears', 'Isaac & Peter Vandorn',
       'Hendrick & Jacob TenEyk', 'Prior & Joy', 'Benezet & Backman',
       'Gilbert Hammond and Cornelius Tommand', 'Simons & Henry',
       'Rotch and Jarvis', 'Moses Dichey and George Dukey',
       'Levy and Levy', 'Peter and Isaac Wikoff',
       'Peter And Isaac Wikoff', 'Milner and Haynes',
       'Michel and Zion Churches', 'Chalone and White', 'Charles & White',
       'Vance Caldwell and Vance', 'Jacob and Henry Geiger',
       'Let

In [85]:
#length of one - then 

#length of two - then

#two one letter - two last names
#first one is one word, second is more than one word - attach last word onto 

#length of three/4 - manually handle

In [86]:
#40857 'Sarah Charman for the use of Gilbert Hammond' 1
#56384' Samuel Harris Esq Mathia Birkly' 2
#'Simon Dreisbach For Estates' 1
#61411 ['Will Robinson', 'Sons']


In [87]:
def parseNames(ind, val):
    #global added_data
    
    if len(val) == 1:
        val = val[0]
        normal_join.loc[ind, 'Last Name 1 '] = val.split(" ")[-1].strip()
        normal_join.loc[ind, 'First Name 1 '] = " ".join(val.split(" ")[:-1]).strip()
    elif len(val) == 2:
        name1 = val[0].strip().split(" ")
        name2 = val[1].strip().split(" ")
        #special cases
        if ind == 40857:
            name1 = 'Gilbert Hammond'
        if ind == 56384:
            name2 = 'Samuel Harris'
        if ind == 61411:
            normal_join.loc[ind, 'Last Name 1 '] = 'Robinson'
            normal_join.loc[ind, 'First Name 1 '] = 'Will'
            normal_join.loc[ind, 'notes'] = 'Sons'
            return
        
        #normal handling
        if len(name1) == 1 and len(name2) == 1:
            
            normal_join.loc[ind, 'notes'] = normal_join.loc[ind, 'First Name 1 ']
            
            normal_join.loc[ind, 'Last Name 1 '] = name1[0]
            normal_join.loc[ind, 'First Name 1 '] = math.nan

            normal_join.loc[ind, 'Last Name 2'] = name2[0]
            normal_join.loc[ind, 'First Name 2'] = math.nan
        elif len(name1) == 1:
            normal_join.loc[ind, 'notes'] = normal_join.loc[ind, 'First Name 1 ']
            
            normal_join.loc[ind, 'Last Name 1 '] = name2[-1].strip()
            normal_join.loc[ind, 'First Name 1 '] = name1[0]

            normal_join.loc[ind, 'Last Name 2'] = name2[-1].strip()
            normal_join.loc[ind, 'First Name 2'] = " ".join(name2[:-1])
        elif len(name2) == 1:
            normal_join.loc[ind, 'notes'] = normal_join.loc[ind, 'First Name 1 ']
            
            normal_join.loc[ind, 'Last Name 1 '] = name2[-1].strip()
            normal_join.loc[ind, 'First Name 1 '] = " ".join(name1)

            normal_join.loc[ind, 'Last Name 2'] = name2[0]
            normal_join.loc[ind, 'First Name 2'] = math.nan
        else:
            normal_join.loc[ind, 'Last Name 1 '] = name1[-1].strip()
            normal_join.loc[ind, 'First Name 1 '] = " ".join(name1[:-1])
            
            normal_join.loc[ind, 'Last Name 2'] = name2[-1].strip()
            normal_join.loc[ind, 'First Name 2'] = " ".join(name2[:-1])
    else:
        if ind == 53730:
            #53730 ['Jn Nixon, I M Nerbitt, Alexander Forster']
            normal_join.loc[ind, 'Last Name 1 '] = "Nixon"
            normal_join.loc[ind, 'First Name 1 '] = "Jn"
            
            normal_join.loc[ind, 'Last Name 2'] = "Nerbitt"
            normal_join.loc[ind, 'First Name 2'] = "I M"
                        
            normal_join.loc[ind, 'Last Name 3'] = "Forster"
            normal_join.loc[ind, 'First Name 3'] = "Alexander"
        elif ind == 57469:
            #57469 ['Hre Rest ', ' Ch ', ' St Pr Chumess']
            normal_join.loc[ind, 'Last Name 1 '] = "Rest"
            normal_join.loc[ind, 'First Name 1 '] = "Hre"
        
            normal_join.loc[ind, 'Last Name 2'] = "Chumess"
            normal_join.loc[ind, 'First Name 2'] = "St Pr"
            
            normal_join.loc[ind, 'Last Name 3'] = "Ch"

In [88]:
for ind, val in joinedNames.items():
    parseNames(ind, val)

In [89]:
two_names.loc[normal_join.index] = normal_join
normal_join_cleaned = two_names.loc[normal_join.index]
#two_names.loc[normal_join.index]

In [90]:
weird_first_names.loc[two_names.index] = two_names

In [91]:
filter_out_names.extend(normal_join.index)

In [92]:
filter_out_names.extend(two_names.index.tolist())
filter_out_names = list(set(filter_out_names))

<h3> Handling treasurers </h3>

In [93]:
#society's treasurer lyme - just society 
#get organization name from words after treasurer

In [94]:
def testString(string, match):
    try:
        return match in string.lower()
    except:
        return False

In [95]:
treasurer = weird_first_names[weird_first_names['First Name 1 '].apply(lambda x: testString(x, "treas") and x not in filter_out_names)]
treasurer.shape

(70, 17)

In [96]:
filter_out_names.extend(treasurer['First Name 1 '].unique().tolist())
treasurer['First Name 1 '].unique()

array(['Ebenezer Storer Treasurer H College',
       'Job Cushing Treasurer for Shrewsbury',
       'Jos Richards Treas 2nd Precinct Roxbury',
       'Dan Thurston Treasurer Church in Franklin',
       'Simeon Howard Treasurer Convention of Ministers',
       'Eli Root Treasurer of Pittsfield', "Society's Treasurer Lyme"],
      dtype=object)

In [97]:
treasurer['original text'] = treasurer['First Name 1 ']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [98]:
#some long code handling treasurer replacements

In [99]:
tempindex = treasurer[treasurer['First Name 1 '].apply(lambda x: x ==  'Ebenezer Storer Treasurer H College')].index
results = ['organization', 'H College', 'treasurer', 'Ebenezer', 'Storer']
treasurer.loc[tempindex, ['Title 1', 'First Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2']] = results

tempindex = treasurer[treasurer['First Name 1 '].apply(lambda x: x ==  'Job Cushing Treasurer for Shrewsbury')].index
results = ['organization', 'Shrewsbury', 'treasurer', 'Job', 'Cushing']
treasurer.loc[tempindex, ['Title 1', 'First Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2']] = results

tempindex = treasurer[treasurer['First Name 1 '].apply(lambda x: x ==  'Jos Richards Treas 2nd Precinct Roxbury')].index
results = ['organization', '2nd Precinct Roxbury', 'treasurer', 'Jos', 'Richards']
treasurer.loc[tempindex, ['Title 1', 'First Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2']] = results

tempindex = treasurer[treasurer['First Name 1 '].apply(lambda x: x ==  'Dan Thurston Treasurer Church in Franklin')].index
results = ['organization', 'Church in Franklin', 'treasurer', 'Dan', 'Thurston']
treasurer.loc[tempindex, ['Title 1', 'First Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2']] = results

tempindex = treasurer[treasurer['First Name 1 '].apply(lambda x: x ==  'Simeon Howard Treasurer Convention of Ministers')].index
results = ['organization', 'Convention of Ministers', 'treasurer', 'Simeon', 'Howard']
treasurer.loc[tempindex, ['Title 1', 'First Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2']] = results

tempindex = treasurer[treasurer['First Name 1 '].apply(lambda x: x ==  'Eli Root Treasurer of Pittsfield')].index
results = ['organization', 'Pittsfield', 'treasurer', 'Eli', 'Root']
treasurer.loc[tempindex, ['Title 1', 'First Name 1 ', 'Title 2', 'First Name 2', 'Last Name 2']] = results

tempindex = treasurer[treasurer['First Name 1 '].apply(lambda x: x ==  "Society's Treasurer Lyme")].index
results = ['organization', 'Society', 'treasurer', 'Lyme']
treasurer.loc[tempindex, ['Title 1', 'First Name 1 ', 'Title 2', 'Last Name 2']] = results

In [100]:
weird_first_names.loc[treasurer.index] = treasurer
treasurer_cleaned = weird_first_names.loc[treasurer.index]
#weird_first_names.loc[treasurer.index]

In [101]:
filter_out_names.extend(treasurer_cleaned.index.tolist())

<h2> Handling everything else </h2>

In [102]:
other_cases = weird_first_names[pd.Series(weird_first_names.index).apply(lambda x: x not in filter_out_names).values]
other_cases.shape

(1662, 17)

In [103]:
nlp = spacy.load("en_core_web_sm")
other_fixed_names = []
for entity in other_cases['First Name 1 ']:
    doc = nlp(entity)
    if len(doc.ents) == 1:
        ent = doc.ents[0]
        if ent.label_ == "PERSON" and ent.text == entity:
            other_fixed_names.append(ent.text)
        elif ent.label == "ORG" and ent.text == entity:
            other_fixed_names.append(ent.text)
        else:
            other_fixed_names.append(False)
    elif len(doc.ents) == 2:
        ent = doc.ents[0]
        ent2 = doc.ents[1]
        if ent.label_ == "Person" and ent2.label_ == "Person" and (ent.text + " " + ent2.text) == entity:
            other_fixed_names.append(ent.text + " " + ent2.text)
        else:
            other_fixed_names.append(False)
    else:
        other_fixed_names.append(False)

In [104]:
other_cases

,State,Year,Month,Day,Title 1,First Name 1,Last Name 1,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text
5804,2,1778,1,1.0,NaN,The Navy Board Eastern Department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600,411.466700,NaN,NaN
5805,2,1778,1,1.0,NaN,The Navy Board Eastern Department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600,411.466700,NaN,NaN
5806,2,1778,1,1.0,NaN,The Navy Board Eastern Department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500,342.888900,NaN,NaN
5807,2,1778,1,1.0,NaN,The Navy Board Eastern Department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500,342.888900,NaN,NaN
5808,2,1778,1,1.0,NaN,The Navy Board Eastern Department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500,342.888900,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80895,9,1780,2,1.0,NaN,Jesse Taylor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300,9.020833,NaN,NaN
80906,9,1779,3,19.0,NaN,William Bradley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000,94.402780,NaN,NaN
80907,9,1779,3,19.0,NaN,Samuel Oldham,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000,94.402780,NaN,NaN
80908,9,1780,1,1.0,NaN,John Hay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400,13.616670,NaN,NaN


In [105]:
#what is william thomas exos james ecum
print(sum([1 if x is False else 0 for x in other_fixed_names]))
#set(other_fixed_names)

435


<h3> Proper names that are just misplaced </h3>

In [106]:
indices = [not x if x is False else False for x in other_fixed_names]
indices_neg = [not x for x in indices]

In [107]:
swap_names = pd.Series(other_fixed_names)[indices_neg]

In [108]:
other_cases.loc[indices_neg, 'First Name 1 '] = swap_names
other_cases.loc[indices_neg, 'Last Name 1 '] = [x.split(" ")[-1] for x in swap_names]
other_cases.loc[indices_neg, 'First Name 1 '] = [" ".join(x.split(" ")[:-1]) for x in swap_names]

In [109]:
weird_first_names.loc[other_cases[indices_neg].index] = other_cases.loc[indices_neg]
weird_first_names.loc[other_cases[indices_neg].index]

,State,Year,Month,Day,Title 1,First Name 1,Last Name 1,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text
10301,2,1779,1,28.0,NaN,John Davis,Williams,NaN,NaN,NaN,NaN,NaN,NaN,500,58.791670,NaN,NaN
34954,5,1779,3,31.0,NaN,John Treat,Crane,NaN,NaN,NaN,NaN,NaN,NaN,500,45.423610,NaN,NaN
35442,5,1779,4,21.0,NaN,Samuel Wright,Hortsham,NaN,NaN,NaN,NaN,NaN,NaN,400,33.983330,NaN,NaN
35798,5,1779,5,4.0,NaN,Abraham Abraham,Haring,NaN,NaN,NaN,NaN,NaN,NaN,400,32.600000,NaN,NaN
35805,5,1779,5,4.0,NaN,Abraham Abraham,Haring,NaN,NaN,NaN,NaN,NaN,NaN,300,24.450000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80895,9,1780,2,1.0,NaN,Jesse,Taylor,NaN,NaN,NaN,NaN,NaN,NaN,300,9.020833,NaN,NaN
80906,9,1779,3,19.0,NaN,William,Bradley,NaN,NaN,NaN,NaN,NaN,NaN,1000,94.402780,NaN,NaN
80907,9,1779,3,19.0,NaN,Samuel,Oldham,NaN,NaN,NaN,NaN,NaN,NaN,1000,94.402780,NaN,NaN
80908,9,1780,1,1.0,NaN,John,Hay,NaN,NaN,NaN,NaN,NaN,NaN,400,13.616670,NaN,NaN


<h2> Handling other weird names/organizations </h2>

In [110]:
other_cases_fixed = pd.read_csv('Data/export_weird_names.csv')

In [111]:
index_indices = other_cases[indices].index

In [112]:
for ind in index_indices:
    fname = other_cases.loc[ind, 'First Name 1 ']
    series = other_cases_fixed[other_cases_fixed['original name'].apply(lambda x: x == fname)].drop('original name', axis = 1).values.flatten().tolist()
    other_cases.loc[ind, ['Title 1','First Name 1 ','Last Name 1 ','Title 2','First Name 2','Last Name 2',
                          'Title 3','First Name 3','Last Name 3','notes','original text']] = series      

In [113]:
other_cases[indices]

,State,Year,Month,Day,Title 1,First Name 1,Last Name 1,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text
5804,2,1778,1,1.0,organization,The Navy Board Eastern Department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600,411.466700,NaN,The Navy Board Eastern Department
5805,2,1778,1,1.0,organization,The Navy Board Eastern Department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600,411.466700,NaN,The Navy Board Eastern Department
5806,2,1778,1,1.0,organization,The Navy Board Eastern Department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500,342.888900,NaN,The Navy Board Eastern Department
5807,2,1778,1,1.0,organization,The Navy Board Eastern Department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500,342.888900,NaN,The Navy Board Eastern Department
5808,2,1778,1,1.0,organization,The Navy Board Eastern Department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500,342.888900,NaN,The Navy Board Eastern Department
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80855,9,1779,3,29.0,NaN,Michl,Stump,NaN,NaN,NaN,NaN,NaN,NaN,300,27.429170,NaN,Michl Stump
80856,9,1779,3,29.0,NaN,Michl,Stump,NaN,NaN,NaN,NaN,NaN,NaN,800,73.144440,NaN,Michl Stump
80858,9,1779,5,16.0,NaN,Captn C,Mossby,NaN,NaN,NaN,NaN,NaN,NaN,49,3.843097,NaN,Captn C Mossby
80860,9,1779,3,9.0,NaN,Fras,Hobday,NaN,NaN,NaN,NaN,NaN,NaN,140,13.646110,NaN,Fras Hobday


In [114]:
weird_first_names.loc[other_cases.index] = other_cases
other_cases_cleaned = weird_first_names.loc[other_cases.index]

## Move weird_first_names back to laons

In [115]:
loans.loc[weird_first_names.index] = weird_first_names

In [116]:
#add additional row that is the same as the first when last name 2 applies
#add index indicating shared

In [117]:
(loans[[not x for x in loans['Last Name 2'].apply(betterIsNan)]][['First Name 1 ', 'Last Name 1 ', 'First Name 2', 'Last Name 2']].drop_duplicates()).shape

(375, 4)

In [118]:
loans[[not x for x in loans['Last Name 2'].apply(betterIsNan)]][['First Name 1 ', 'Last Name 1 ', 'First Name 2', 'Last Name 2']].shape

(3178, 4)

In [119]:
#checking if there are any names in first name column that are two first names
#check confirmed - also shows that there are no NA first name or last name columns
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(loans[[not x for x in loans['Last Name 2'].apply(betterIsNan)]][['First Name 1 ', 'Last Name 1 ', 'First Name 2', 'Last Name 2']].drop_duplicates())

,First Name 1,Last Name 1,First Name 2,Last Name 2
2253,Abraham,Livingston,William,Turnbull
2428,NaN,Lee,NaN,Jones
2759,NaN,Clark,NaN,Nightingale
2822,NaN,Pitts,NaN,Call
3644,NaN,Bryant,NaN,Dennie
3749,Sam,Hasting,,Austin
3807,NaN,Hubbart,NaN,Greene
4000,Sarah,Green,John,Gray
4017,John,Gray,Thomas,Dawes
4233,NaN,Hunt,NaN,Sherburne


## Final Manual Replacement

In [120]:
repl_index = loans[loans['First Name 1 '].apply(lambda x: not pd.isnull(x) and x == 'Peter & Anna Maricha')].index
loans.loc[repl_index, 'original text'] = 'Peter & Anna Maricha'
loans.loc[repl_index, ['First Name 1 ', 'Last Name 1 ', 'First Name 2', 'Last Name 2']] = ['Peter', 'Maricha', 'Anna', 'Maricha']
repl_index = loans[loans['First Name 1 '].apply(lambda x: not pd.isnull(x) and x == 'Myndert & Mary')].index
loans.loc[repl_index, 'original text'] = 'Myndert & Mary Van Schaick'
loans.loc[repl_index, ['First Name 1 ', 'Last Name 1 ', 'First Name 2', 'Last Name 2']] = ['Myndert', 'Van Schaick', 'Mary', 'Van Schaick']

In [121]:
repl_index = loans[[not pd.isnull(x) and '&' in x and pd.isnull(y) for x, y in zip(loans['First Name 1 '], loans['Title 1'])]].index
loans.loc[repl_index, 'original text'] = [x + " " + y for x, y in zip(loans.loc[repl_index, 'First Name 1 '], loans.loc[repl_index, 'Last Name 1 '])]
loans.loc[repl_index, ['Last Name 2']] = loans.loc[repl_index, 'Last Name 1 ']
loans.loc[repl_index, ['Last Name 1 ']] = loans.loc[repl_index, 'First Name 1 '].apply(lambda x: x.replace("&", "")).tolist()
loans.loc[repl_index, ['First Name 1 ']] = [np.nan] * len(repl_index)

In [122]:
repl_index = loans[loans['Last Name 1 '].apply(lambda x: not pd.isnull(x) and '&' in x)].index
loans.loc[repl_index, 'original text'] = [x + " " + y for x, y in zip(loans.loc[repl_index, 'First Name 1 '], loans.loc[repl_index, 'Last Name 1 '])]
loans.loc[repl_index, ['First Name 1 ']] = 'Samuel'
loans.loc[repl_index, ['Last Name 1 ']] = 'Samuel'
loans.loc[repl_index, ['notes']] = 'son'

In [123]:
repl_index = loans[loans['original text'].apply(lambda x: x == 'Sam Austin exec to Sam Hasting')].index 
loans.loc[repl_index, ['First Name 2']] = 'Sam'

In [124]:
repl_index = loans[loans['Last Name 1 '].apply(lambda x: x == 'Gordon Treasurer to the Convention of Ministers')].index
loans.loc[repl_index, ['Title 1', 'Last Name 1 ', 'notes', 'original text']] = ['Treasurer', 'Gordon', 'Convention of Ministers', 
                                                                               'William Gordon Treasurer to the Convention of Ministers']
repl_index = loans[loans['Last Name 1 '].apply(lambda x: x == 'Thurston Treas Church Wrentham')].index
loans.loc[repl_index, ['Title 1', 'Last Name 1 ', 'notes', 'original text']] = ['Treasurer', 'Thurston', 'Church Wrentham', 
                                                                                'Daniel Thurston Treas Church Wrentham']
repl_index = loans[loans['Last Name 1 '].apply(lambda x: x == 'Biglow Guardian to her Children')].index
loans.loc[repl_index, ['Title 1', 'Last Name 1 ', 'notes', 'original text']] = ['Guardian', 'Biglow', 'children', 
                                                                                'Anna Biglow Guardian to her Children']
repl_index = loans[loans['Last Name 1 '].apply(lambda x: x == 'Miller Treas Westminster')].index
loans.loc[repl_index, ['Title 1', 'Last Name 1 ', 'notes', 'original text']] = ['Treas', 'Miller', 'Westminster', 
                                                                                'Joseph Miller Treas Westminster']
repl_index = loans[loans['Last Name 1 '].apply(lambda x: x == 'Allen Executor to B Winchester')].index
loans.loc[repl_index, ['Title 1', 'Last Name 1 ', 'First Name 2', 'Last Name 2', 'original text']] = ['Executor', 'Allen', 'B', 'Winchester', 
                                                                                                      'Joseph Allen Executor to B Winchester']
repl_index = loans[loans['Last Name 1 '].apply(lambda x: x == 'Estates')].index 
loans.loc[repl_index, ['First Name 1 ', 'Last Name 1 ', 'notes']] = ['Simon', 'Dreisbach', 'Estates']
repl_index = loans[loans['Last Name 1 '].apply(lambda x: x == 'Comp.')].index 
loans.loc[repl_index, ['First Name 1 ', 'Last Name 1 ', 'notes']] = ['SD', 'George', 'Comp.']

In [133]:
temp = loans[[pd.isnull(x) and pd.isnull(y) and pd.isnull(z) for x, y, z in zip(loans['First Name 1 '], loans['First Name 2'], loans['First Name 3'])]]

In [183]:
temp['original text'].apply(lambda x: x.replace(' and ', ' & ') if not pd.isnull(x) else x)

374                    NaN
414                    NaN
492                    NaN
523                    NaN
524                    NaN
               ...        
80900       Doerner & Mark
80901       Doerner & Mark
80902       Doerner & Mark
80903       Doerner & Mark
80904    Mitchell & Oldham
Name: original text, Length: 1385, dtype: object

In [ ]:
#swamp blake & homans and homans & blake

In [180]:
temp[['original text', 'State']].drop_duplicates().to_csv("state_companies.csv")

In [178]:
loans[loans['Last Name 1 '].apply(lambda x: not pd.isnull(x) and 'blackman' == x.lower())][['First Name 1 ','Last Name 1 ']].drop_duplicates()

,First Name 1,Last Name 1
20532,Lachariah,Blackman
21239,Ebenezer,Blackman
56041,Andrew,Blackman


In [135]:
loans.to_csv('Data/Pre1790/cleaned/loan_office_certificates_9_states_cleaned.csv')